<a href="https://colab.research.google.com/github/gustavo-duarte-silva/ML-Vendas-Casas-Cuiaba/blob/main/Previsao_Casas_Cuiaba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Importanto Bibliotecas Padrão para Analise de dados
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#dataset de vendas de casas no bairro Jardim Imperial - Cuiabá -MT
df = pd.read_csv('/content/drive/MyDrive/CIENTISTA DE DADOS/PROJETOS/JARDIM CUIABA/Data.csv')

In [ ]:
#Dataset Obtido no Kaggle - Dataset Muito Pequeno para realizar o Train_Test_Split
df.head()

,preco,endereco,tamanhom2,quartos,banheiros,suite,garagem,iptu,condominio
0,1.380.000,"Avenida das Palmeiras - Jardim Imperial, Cuiab...",321,3,3.0,3.0,4,550.0,550.0
1,430.000,"Avenida das Palmeiras, 7 - Jardim Imperial, Cu...",88,3,2.0,1.0,2,NaN,NaN
2,370.000,"Avenida das Palmeiras, 20 - Jardim Imperial, C...",89,3,2.0,1.0,2,NaN,NaN
3,420.000,"Avenida das Palmeiras, 004 - Jardim Imperial, ...",132,3,2.0,1.0,\n,NaN,NaN
4,365.000,"Avenida das Palmeiras, 100 - Jardim Imperial, ...",90,3,2.0,1.0,2,NaN,NaN


In [ ]:
df.shape

(106, 9)

In [ ]:
#Valores faltantes 
colunas = ['tamanhom2','quartos','banheiros','suite','garagem','iptu','condominio']
print("Valores Null")
for name in colunas:
  print(name)
  print(df[name].isnull().value_counts())


Valores Null
tamanhom2
False    106
Name: tamanhom2, dtype: int64
quartos
False    106
Name: quartos, dtype: int64
banheiros
False    105
True       1
Name: banheiros, dtype: int64
suite
False    87
True     19
Name: suite, dtype: int64
garagem
False    106
Name: garagem, dtype: int64
iptu
True     79
False    27
Name: iptu, dtype: int64
condominio
True     79
False    27
Name: condominio, dtype: int64


Temos Valores Faltantes:
* Banheiro: 1
* Suite: 19
* IPTU: 27
* Condominio: 27

In [ ]:
#tipos dos dados
df.dtypes

preco          object
endereco       object
tamanhom2       int64
quartos        object
banheiros     float64
suite         float64
garagem        object
iptu          float64
condominio    float64
dtype: object

In [ ]:
def pre_processamento(df):
  dff = df.copy()
  dff = dff.drop(columns=['iptu','condominio','preco','endereco'], axis=0)
  dff['quartos'] = dff.quartos.replace('2 a 3\n', np.NaN)
  dff['quartos'] = dff.quartos.replace('1 até 2\n', np.NaN)
  dff['quartos'] = dff.quartos.replace('\n                ', np.NaN)
  dff['garagem'] = dff.garagem.replace('\n                ', np.NaN)
  dff['garagem'] = dff.garagem.replace('1 até 2\n', np.NaN)
  dff['quartos'].fillna(dff['quartos'].median(), inplace = True)
  dff['suite'].fillna(dff['suite'].median(), inplace = True)
  dff['garagem'].fillna(dff['garagem'].median(), inplace = True)
  dff['banheiros'].fillna(dff['banheiros'].median(), inplace = True)
  dff['quartos'] = dff.quartos.astype('int')
  dff['banheiros'] = dff.banheiros.astype('int')
  dff['suite'] = dff.suite.astype('int')
  dff['garagem'] = dff.garagem.astype('int')
  

  return dff

In [ ]:
x = pre_processamento(df)
y = df.iloc[:,0].str.replace('.','').astype('float')

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
#GRID SEARCH
from sklearn.model_selection import GridSearchCV
class gridSearchAll():
    
    def __init__(self, scoring, num_folds):
        self.grid_models = []
        self.scoring = scoring
        self.num_folds = num_folds
        self.best = []
    
    def insert_model(self, estimator_base, param_grid):
        self.grid_models.append([estimator_base, param_grid])
    def fit_all(self, X, y):
      for est, param in self.grid_models:
        grid = GridSearchCV(estimator = est, 
                    param_grid = param, 
                    scoring = self.scoring, 
                    cv = self.num_folds)
        grid.fit(X, y)
        self.best.append(grid)

    def best_all_grid_models(self):
      best_models = []
      for grid in self.best:
        best_models.append(grid.best_estimator_)
      print("Melhores Modelos:")
      print(best_models)

In [ ]:
mygrid = gridSearchAll(scoring='r2',num_folds=5)

In [ ]:
#Estimadores
knn = KNeighborsRegressor()
dtr =  DecisionTreeRegressor()
#Params
params_knn = {"n_neighbors":[1,2,3,4,5,6]}
params_dtr = {"max_depth":[1,2,5,6]}

In [ ]:
mygrid.insert_model(knn, params_knn)
mygrid.insert_model(dtr, params_dtr)

In [ ]:
mygrid.grid_models

[[KNeighborsRegressor(), {'n_neighbors': [1, 2, 3, 4, 5, 6]}],
 [DecisionTreeRegressor(), {'max_depth': [1, 2, 5, 6]}]]

In [ ]:
mygrid.fit_all(x, y)

In [ ]:
mygrid.best_all_grid_models()

Melhores Modelos:
[KNeighborsRegressor(n_neighbors=1), DecisionTreeRegressor(max_depth=6)]


In [ ]:
#Escolhendo a algoritimo DecisionTreeRegressor
modelo_DTR = DecisionTreeRegressor(max_depth=6)

In [ ]:
modelo_DTR.fit(x, y)

DecisionTreeRegressor(max_depth=6)

In [ ]:
ypred = modelo_DTR.predict(x)

In [ ]:
r2_score(y_true=y, y_pred = ypred)

0.9945054220472815